# Get Target

In [ ]:
!pip install -r requirements_colab.txt -q

> To speed up the review process , i provided the ***drive id*** of the data i've created from the Train creation folder noteboooks .
---
> I  also add each data drive link in the Readme Pdf file attached with this solution
---
> The data Used in this notebook is **S2TrainObs1**

In [ ]:
!gdown --id 1hNRbtcqd9F6stMOK1xAZApDITwAjiSDJ

Downloading...
From: https://drive.google.com/uc?id=1hNRbtcqd9F6stMOK1xAZApDITwAjiSDJ
To: /content/S2TrainObs1.csv
49.8MB [00:00, 137MB/s] 


In [ ]:
#import necessary dependecies
import os
import warnings
import numpy as np  
import pandas as pd
import random
import gc
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold

from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

warnings.filterwarnings('ignore') 
np.random.seed(111)
random.seed(111)

In [ ]:
def create_target():
  train =pd.read_csv("S2TrainObs1.csv" )

  train = train.groupby('field_id').median().reset_index().sort_values('field_id')
  train.label = train.label.astype('int')
  return train[['field_id']]

# Optimized Blend

In [ ]:
y_true = pd.get_dummies(create_target()['label']).values

In [ ]:
test_dict = {'Catboost': 'S1_Catboost.csv', 
            'LightGbm': 'S1_LGBM.csv', 
            'Xgboost': 'S1_XGBOOST.csv', 
            'NN Attention': 'S1_NNAttention.csv' ,
            'Neural Network' : 'S1_NN.csv',
             
            'CatboostS2': 'S2_Catboost.csv', 
            'XgboostS2' : 'S2_Xgboost.csv', 
            'NN_AttentionS2': 'S2_NNAttention.csv' ,
            'Neural_NetworkS2' : 'S2_NN.csv',
           }

BlendTest = np.zeros((len(test_dict), 35295,y_true.shape[1]))
for i in range(BlendTest.shape[0]):
    BlendTest[i] = pd.read_csv(list(test_dict.values())[i]).values[:,1:]

In [ ]:
BlendPreds = np.tensordot([0.175 , 0.2275, 0.1225, 0.0875, 0.0875,0.0975, 0.0825, 0.06  , 0.06 ], BlendTest, axes = ((0), (0)))

In [ ]:
BlendPreds.shape

(35295, 9)

In [ ]:
pred_df = pd.DataFrame(BlendPreds)
pred_df = pred_df.rename(columns={
    0:'Crop_ID_1',
    1:'Crop_ID_2', 
    2:'Crop_ID_3',
    3:'Crop_ID_4',
    4:'Crop_ID_5',
    5:'Crop_ID_6',
    6:'Crop_ID_7',
    7:'Crop_ID_8',
    8:'Crop_ID_9'
})
pred_df['field_id'] = pd.read_csv("S1_10FoldsCatboost_V5.csv")['field_id'].astype('int').values
pred_df = pred_df[['field_id', 'Crop_ID_1', 'Crop_ID_2', 'Crop_ID_3', 'Crop_ID_4', 'Crop_ID_5', 'Crop_ID_6', 'Crop_ID_7', 'Crop_ID_8', 'Crop_ID_9']]
pred_df.head()

,field_id,Crop_ID_1,Crop_ID_2,Crop_ID_3,Crop_ID_4,Crop_ID_5,Crop_ID_6,Crop_ID_7,Crop_ID_8,Crop_ID_9
0,30,0.057399,0.310956,0.006510,0.000065,0.169113,0.349293,1.039147e-01,1.862869e-03,0.000885
1,39,0.829348,0.115107,0.022136,0.013806,0.016729,0.002758,6.609615e-05,4.032507e-05,0.000010
2,49,0.037060,0.394246,0.000983,0.000621,0.001637,0.510139,5.050189e-02,3.582930e-03,0.001229
3,54,0.000004,0.000041,0.000061,0.999787,0.000101,0.000002,3.262132e-07,1.782767e-08,0.000002
4,56,0.785304,0.032771,0.000954,0.000332,0.001270,0.053306,1.229265e-01,3.114503e-03,0.000022


# Optimized Stacking

In [ ]:
y_true = create_target()

oof_dict = {'Catboost': 'S1_oof_cat.npy', 
            'LightGbm': 'S1_oof_lgbm.npy', 
            'Xgboost': 'S1_oof_XGBOOST.npy', 
            'NN Attention': 'S1_oof_NNAttention.npy' ,
            'Neural Network' : 'S1_oof_NN.npy',

            'CatboostS2': 'S2_oof_cat.npy', 
            'XgboostS2' : 'S2_oof_xgb.npy', 
            'NN_AttentionS2'   : 'S2_oof_NNAttention.npy' ,
            'Neural_NetworkS2' : 'S2_NN.npy',
           }

oof_data = y_true.copy()
for i in range(len(list(oof_dict.values())) ):
    local = pd.DataFrame(np.load(list(oof_dict.values())[i]),columns=[f'{list(oof_dict.keys())[i]}_oof_{j}' for j in range(1,10)] )
    oof_data = pd.concat([oof_data,local],axis=1)

In [ ]:
oof_data.head()

,field_id,label,Catboost_oof_1,Catboost_oof_2,Catboost_oof_3,Catboost_oof_4,Catboost_oof_5,Catboost_oof_6,Catboost_oof_7,Catboost_oof_8,Catboost_oof_9,LightGbm_oof_1,LightGbm_oof_2,LightGbm_oof_3,LightGbm_oof_4,LightGbm_oof_5,LightGbm_oof_6,LightGbm_oof_7,LightGbm_oof_8,LightGbm_oof_9,Xgboost_oof_1,Xgboost_oof_2,Xgboost_oof_3,Xgboost_oof_4,Xgboost_oof_5,Xgboost_oof_6,Xgboost_oof_7,Xgboost_oof_8,Xgboost_oof_9,NN Attention_oof_1,NN Attention_oof_2,NN Attention_oof_3,NN Attention_oof_4,NN Attention_oof_5,NN Attention_oof_6,NN Attention_oof_7,NN Attention_oof_8,NN Attention_oof_9,Neural Network_oof_1,Neural Network_oof_2,...,Neural Network_oof_6,Neural Network_oof_7,Neural Network_oof_8,Neural Network_oof_9,CatboostS2_oof_1,CatboostS2_oof_2,CatboostS2_oof_3,CatboostS2_oof_4,CatboostS2_oof_5,CatboostS2_oof_6,CatboostS2_oof_7,CatboostS2_oof_8,CatboostS2_oof_9,XgboostS2_oof_1,XgboostS2_oof_2,XgboostS2_oof_3,XgboostS2_oof_4,XgboostS2_oof_5,XgboostS2_oof_6,XgboostS2_oof_7,XgboostS2_oof_8,XgboostS2_oof_9,NN_AttentionS2_oof_1,NN_AttentionS2_oof_2,NN_AttentionS2_oof_3,NN_AttentionS2_oof_4,NN_AttentionS2_oof_5,NN_AttentionS2_oof_6,NN_AttentionS2_oof_7,NN_AttentionS2_oof_8,NN_AttentionS2_oof_9,Neural_NetworkS2_oof_1,Neural_NetworkS2_oof_2,Neural_NetworkS2_oof_3,Neural_NetworkS2_oof_4,Neural_NetworkS2_oof_5,Neural_NetworkS2_oof_6,Neural_NetworkS2_oof_7,Neural_NetworkS2_oof_8,Neural_NetworkS2_oof_9
0,0.0,5,0.192856,0.410211,0.127925,0.074467,0.131364,0.023724,0.023251,0.000336,1.586557e-02,0.102729,0.664125,0.065286,0.051717,0.054234,0.016235,0.034760,0.000457,1.045802e-02,0.214095,0.456742,0.160001,0.059933,0.050241,0.030939,0.013279,0.000322,1.444889e-02,0.025384,0.604382,0.050536,0.091948,0.084124,0.025097,0.040424,7.828907e-04,7.732173e-02,1.160256e-02,0.271368,...,0.042576,6.743674e-02,1.106742e-05,2.863588e-02,0.151103,0.408235,0.153073,0.057755,0.140192,0.015172,0.061491,0.001563,1.141486e-02,0.055040,0.528661,0.139965,0.026667,0.205064,0.015499,0.014682,0.000331,1.409152e-02,0.217523,0.254941,0.024358,0.000888,0.116820,0.069534,0.159333,8.371037e-03,1.482335e-01,0.248323,0.063602,0.165253,0.116913,0.085659,0.039060,0.071560,0.068990,1.406386e-01
1,1.0,4,0.000334,0.010251,0.001398,0.985995,0.001303,0.000640,0.000054,0.000024,9.199984e-08,0.000069,0.001729,0.000345,0.997286,0.000494,0.000064,0.000008,0.000006,1.898847e-07,0.000056,0.000468,0.000293,0.998821,0.000310,0.000045,0.000003,0.000004,8.709280e-08,0.000023,0.000861,0.000027,0.999072,0.000008,0.000004,0.000003,5.015906e-07,3.440801e-08,3.907265e-07,0.000016,...,0.000024,1.361261e-07,2.512553e-08,2.086016e-09,0.000374,0.004996,0.001850,0.991610,0.000780,0.000336,0.000034,0.000019,3.108247e-08,0.000198,0.002835,0.000366,0.995879,0.000392,0.000285,0.000026,0.000019,2.579816e-07,0.000007,0.000595,0.000316,0.999062,0.000009,0.000009,0.000002,5.175813e-08,2.387890e-08,0.000004,0.000168,0.000199,0.999556,0.000006,0.000055,0.000010,0.000001,2.205914e-08
2,2.0,7,0.028638,0.082376,0.005594,0.000415,0.000840,0.260912,0.611905,0.009005,3.152203e-04,0.080072,0.121328,0.003423,0.001126,0.001740,0.140626,0.636485,0.014523,6.770333e-04,0.065643,0.043500,0.002528,0.001089,0.004477,0.087274,0.793377,0.001509,6.028328e-04,0.233084,0.017915,0.002057,0.000428,0.000026,0.501747,0.209540,3.516322e-02,3.971763e-05,4.669775e-02,0.027987,...,0.576060,2.737575e-01,5.741664e-02,4.504967e-06,0.066400,0.236450,0.010485,0.002395,0.000971,0.194929,0.484263,0.003998,1.081458e-04,0.117672,0.088883,0.024491,0.004161,0.004631,0.282346,0.472852,0.002335,2.628644e-03,0.222095,0.062763,0.051674,0.000074,0.013310,0.392390,0.170866,7.996373e-02,6.863958e-03,0.041555,0.017205,0.007484,0.001634,0.000168,0.120018,0.711179,0.100295,4.616875e-04
3,3.0,6,0.004145,0.003187,0.000142,0.000245,0.000001,0.668482,0.323358,0.000439,2.938757e-07,0.003217,0.001539,0.000049,0.000117,0.000036,0.836821,0.157492,0.000722,6.502128e-06,0.002575,0.001577,0.000067,0.000246,0.000031,0.915431,0.077975,0.002092,5.451056e-06,0.006514,0.016649,0.0

In [ ]:
test_dict = {'Catboost': 'S1_Catboost.csv', 
            'LightGbm': 'S1_LGBM.csv', 
            'Xgboost': 'S1_XGBOOST.csv', 
            'NN Attention': 'S1_NNAttention.csv' ,
            'Neural Network' : 'S1_NN.csv',
             
            'CatboostS2': 'S2_Catboost.csv', 
            'XgboostS2' : 'S2_Xgboost.csv', 
            'NN_AttentionS2': 'S2_NNAttention.csv' ,
            'Neural_NetworkS2' : 'S2_NN.csv',
           }

test_data = pd.read_csv("S1_10FoldsCatboost_V5.csv")[['field_id']]
for i in range(len(list(test_dict.values()))):
    local = pd.DataFrame(pd.read_csv(list(test_dict.values())[i]).iloc[:,1:].values,columns=[f'{list(test_dict.keys())[i]}_oof_{j}' for j in range(1,10)] )
    test_data = pd.concat([test_data,local],axis=1)

In [ ]:
test_data.head()

,field_id,Catboost_oof_1,Catboost_oof_2,Catboost_oof_3,Catboost_oof_4,Catboost_oof_5,Catboost_oof_6,Catboost_oof_7,Catboost_oof_8,Catboost_oof_9,LightGbm_oof_1,LightGbm_oof_2,LightGbm_oof_3,LightGbm_oof_4,LightGbm_oof_5,LightGbm_oof_6,LightGbm_oof_7,LightGbm_oof_8,LightGbm_oof_9,Xgboost_oof_1,Xgboost_oof_2,Xgboost_oof_3,Xgboost_oof_4,Xgboost_oof_5,Xgboost_oof_6,Xgboost_oof_7,Xgboost_oof_8,Xgboost_oof_9,NN Attention_oof_1,NN Attention_oof_2,NN Attention_oof_3,NN Attention_oof_4,NN Attention_oof_5,NN Attention_oof_6,NN Attention_oof_7,NN Attention_oof_8,NN Attention_oof_9,Neural Network_oof_1,Neural Network_oof_2,Neural Network_oof_3,...,Neural Network_oof_6,Neural Network_oof_7,Neural Network_oof_8,Neural Network_oof_9,CatboostS2_oof_1,CatboostS2_oof_2,CatboostS2_oof_3,CatboostS2_oof_4,CatboostS2_oof_5,CatboostS2_oof_6,CatboostS2_oof_7,CatboostS2_oof_8,CatboostS2_oof_9,XgboostS2_oof_1,XgboostS2_oof_2,XgboostS2_oof_3,XgboostS2_oof_4,XgboostS2_oof_5,XgboostS2_oof_6,XgboostS2_oof_7,XgboostS2_oof_8,XgboostS2_oof_9,NN_AttentionS2_oof_1,NN_AttentionS2_oof_2,NN_AttentionS2_oof_3,NN_AttentionS2_oof_4,NN_AttentionS2_oof_5,NN_AttentionS2_oof_6,NN_AttentionS2_oof_7,NN_AttentionS2_oof_8,NN_AttentionS2_oof_9,Neural_NetworkS2_oof_1,Neural_NetworkS2_oof_2,Neural_NetworkS2_oof_3,Neural_NetworkS2_oof_4,Neural_NetworkS2_oof_5,Neural_NetworkS2_oof_6,Neural_NetworkS2_oof_7,Neural_NetworkS2_oof_8,Neural_NetworkS2_oof_9
0,30,0.107095,0.262932,0.006750,0.000002,0.168061,0.301674,1.524572e-01,7.034903e-04,3.263414e-04,3.794846e-02,0.353952,0.003963,0.000027,0.104537,4.019422e-01,9.611233e-02,1.066878e-03,4.519848e-04,0.047739,0.353477,0.003346,0.000026,0.109481,3.543548e-01,1.301281e-01,1.056439e-03,3.920414e-04,0.055811,0.297380,0.011131,0.000204,0.173487,2.992985e-01,1.538131e-01,7.317784e-03,1.557601e-03,0.046520,0.328960,0.006543,...,0.362892,3.914557e-02,1.072579e-03,0.001784,0.099415,0.191492,0.008146,0.000002,0.265454,2.954184e-01,1.377500e-01,1.703509e-03,6.185369e-04,2.686587e-02,0.278273,0.005266,0.000036,0.226852,3.972440e-01,6.204018e-02,2.918740e-03,5.045718e-04,0.041120,0.309569,0.009097,0.000099,0.157301,0.451127,0.026184,2.467516e-03,0.003034,1.409206e-02,0.435187,0.011612,0.000229,0.244402,2.510796e-01,4.040882e-02,1.308897e-03,1.680860e-03
1,39,0.882075,0.089919,0.013722,0.005986,0.007382,0.000896,1.313991e-05,7.329836e-06,5.387162e-08,8.697382e-01,0.076377,0.023712,0.012244,0.015819,1.933320e-03,7.951311e-05,8.633358e-05,1.031353e-05,0.866878,0.092401,0.022187,0.006570,0.010279,1.566196e-03,6.935475e-05,4.368999e-05,5.865245e-06,0.768878,0.168540,0.015459,0.015719,0.021324,9.927223e-03,1.141302e-04,2.858439e-05,1.057026e-05,0.686957,0.241258,0.019341,...,0.005650,7.911746e-05,1.481858e-05,0.000031,0.859933,0.077197,0.030461,0.018503,0.012923,9.373837e-04,1.386322e-05,3.270161e-05,1.548373e-08,8.399638e-01,0.086238,0.041110,0.017728,0.014079,7.329827e-04,6.706321e-05,7.469813e-05,6.926843e-06,0.710284,0.205948,0.010874,0.023593,0.046379,0.002728,0.000153,7.473998e-06,0.000033,7.964015e-01,0.130348,0.026051,0.025137,0.017122,4.849683e-03,7.024565e-05,7.537185e-06,1.236286e-05
2,49,0.034494,0.408732,0.000852,0.000320,0.001234,0.513660,3.930285e-02,9.638408e-04,4.405246e-04,5.271837e-02,0.333789,0.001039,0.000792,0.001927,5.516325e-01,5.470992e-02,1.517502e-03,1.875038e-03,0.023400,0.453448,0.001105,0.001306,0.001472,4.703945e-01,4.291718e-02,3.531716e-03,2.426325e-03,0.008941,0.397148,0.000055,0.000038,0.000510,5.465559e-01,3.995302e-02,6.780147e-03,1.868593e-05,0.023400,0.414496,0.000069,...,0.454757,9.625371e-02,7.462219e-03,0.000587,0.054238,0.406445,0.002309,0.000810,0.002537,4.616461e-01,6.756595e-02,3.637855e-03,8.122070e-04,6.760062e-02,0.387984,0.002664,0.001403,0.001759,4.731688e-01,5.660396e-02,5.306844e-03,3.509160e-03,0.021883,0.448379,0.000042,0.000026,0.000337,0.498564,0.026004,4.699463e-03,0.000066,1.926275e-02,0.361248,0.000067,0.000067,0.001765,5.925541e-01,1.973689e-02,5.261837e-03,3.797240e-05
3,54,0.000011,0.000006,

# Stacker-Model

In [ ]:
class AzerStacking :
  def __init__(self,y) :
    self.y = y
  
  def StackingRegressor(self ,KFOLD,stacking_train : pd.DataFrame , stacking_test : pd.DataFrame) :
      
      cols = stacking_train.drop(['field_id','label'],1).columns.tolist()
      X , y , Test = stacking_train[cols] , stacking_train['label'] , stacking_test[cols]
      final_preds = [] ; err_cb = []
      oof_stack = np.zeros((len(X),9)) ;
      
      for fold,(train_index, test_index) in enumerate(KFOLD.split(X,y)):
          X_train, X_test = X.values[train_index], X.values[test_index]
          y_train, y_test = y.values[train_index], y.values[test_index]

          model1 = LGBMClassifier(verbose=10)
          model2 = CatBoostClassifier(iterations=50,verbose=0)
          
          model1.fit(X_train,y_train)
          model2.fit(X_train,y_train)
          
          preds1=model1.predict_proba(X_test)
          preds2=model2.predict_proba(X_test)
          
          preds = preds1*0.7+preds2*0.3
          oof_stack[test_index] = preds
          err_cb.append(log_loss(y_test,preds))
          print(f'logloss fold-{fold+1}/10',log_loss(y_test,preds))

          test_pred1 = model1.predict_proba(Test.values)
          test_pred2 = model2.predict_proba(Test.values)
          test_pred = test_pred1*0.7+test_pred2*0.3
          final_preds.append(test_pred)
      
      print(2*'--------------------------------------')
      print('STACKING Log Loss',log_loss(y, oof_stack))

      return oof_stack,np.mean(final_preds,axis=0)

In [ ]:
folds = StratifiedKFold(n_splits=10,shuffle=True,random_state=47) 
AzerStacker = AzerStacking(y=oof_data['label'])

In [ ]:
oof_stack,stack_preds  = AzerStacker.StackingRegressor(KFOLD=folds ,stacking_train=oof_data ,stacking_test=test_data)

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

logloss fold-1/10 0.5506146441938404
logloss fold-2/10 0.5403484731979808
logloss fold-3/10 0.5302137469754669
logloss fold-4/10 0.5565312589883131
logloss fold-5/10 0.5453391243568311
logloss fold-6/10 0.5371695739757446
logloss fold-7/10 0.5496591540427179
logloss fold-8/10 0.5447447600613889
logloss fold-9/10 0.5357727188452667
logloss fold-10/10 0.5510197693367598
----------------------------------------------------------------------------
STACKING Log Loss 0.5441413355162611


In [ ]:
pred_df = pd.DataFrame(stack_preds)
pred_df = pred_df.rename(columns={
    0:'Crop_ID_1',
    1:'Crop_ID_2', 
    2:'Crop_ID_3',
    3:'Crop_ID_4',
    4:'Crop_ID_5',
    5:'Crop_ID_6',
    6:'Crop_ID_7',
    7:'Crop_ID_8',
    8:'Crop_ID_9'
})
pred_df['field_id'] = pd.read_csv("S1_10FoldsCatboost_V5.csv")['field_id'].astype('int').values
pred_df = pred_df[['field_id', 'Crop_ID_1', 'Crop_ID_2', 'Crop_ID_3', 'Crop_ID_4', 'Crop_ID_5', 'Crop_ID_6', 'Crop_ID_7', 'Crop_ID_8', 'Crop_ID_9']]
pred_df.head()

,field_id,Crop_ID_1,Crop_ID_2,Crop_ID_3,Crop_ID_4,Crop_ID_5,Crop_ID_6,Crop_ID_7,Crop_ID_8,Crop_ID_9
0,30,0.057252,0.243205,0.006306,0.001232,0.263744,0.335978,0.087610,0.003150,0.001522
1,39,0.832992,0.096529,0.029490,0.014661,0.022348,0.002649,0.000899,0.000243,0.000189
2,49,0.044318,0.334390,0.001945,0.001447,0.002621,0.551651,0.058898,0.003952,0.000778
3,54,0.000272,0.000877,0.000970,0.992893,0.000491,0.000174,0.004280,0.000018,0.000026
4,56,0.735058,0.033514,0.002659,0.001098,0.002328,0.079062,0.140079,0.005929,0.000273


# Stacking - Blend 

In [ ]:
pred_df = pd.DataFrame(stack_preds*0.7+ BlendPreds*0.3)
pred_df = pred_df.rename(columns={
    0:'Crop_ID_1',
    1:'Crop_ID_2', 
    2:'Crop_ID_3',
    3:'Crop_ID_4',
    4:'Crop_ID_5',
    5:'Crop_ID_6',
    6:'Crop_ID_7',
    7:'Crop_ID_8',
    8:'Crop_ID_9'
})
pred_df['field_id'] = pd.read_csv("S1_10FoldsCatboost_V5.csv")['field_id'].astype('int').values
pred_df = pred_df[['field_id', 'Crop_ID_1', 'Crop_ID_2', 'Crop_ID_3', 'Crop_ID_4', 'Crop_ID_5', 'Crop_ID_6', 'Crop_ID_7', 'Crop_ID_8', 'Crop_ID_9']]
pred_df.head()

,field_id,Crop_ID_1,Crop_ID_2,Crop_ID_3,Crop_ID_4,Crop_ID_5,Crop_ID_6,Crop_ID_7,Crop_ID_8,Crop_ID_9
0,30,0.057296,0.263530,0.006367,0.000882,0.235355,0.339973,0.092501,0.002764,0.001331
1,39,0.831899,0.102102,0.027284,0.014404,0.020662,0.002681,0.000649,0.000182,0.000135
2,49,0.042141,0.352347,0.001657,0.001199,0.002326,0.539198,0.056379,0.003841,0.000913
3,54,0.000192,0.000626,0.000697,0.994962,0.000374,0.000122,0.002996,0.000013,0.000019
4,56,0.750132,0.033291,0.002147,0.000868,0.002011,0.071335,0.134933,0.005084,0.000198


In [ ]:
# Write the predicted probabilites to a csv for submission
pred_df.to_csv('S1_Stacking_Blending.csv', index=False)